In [126]:
import os
import scipy.io
import numpy as np
from run import RunJupyterExperiment
import data_extractor
from glob import glob

# Section1: run experiment

You need to sepcify your paths in this block.

In [2]:
experiment_history_folder = 'experiment_history/'
training_folder_path = './dataset/bach_dataset/'
test_folder_path = './dataset/shanx_dataset/'

You don't need to change anything in the following block.

In [3]:
configurations = {
    'experiment_history_folder':experiment_history_folder,
    'train_test_path':[
    training_folder_path,
    test_folder_path, 
    ],
    'trainp':None,
}



In [4]:
RunJupyterExperiment(configurations)

using distinct trainning and test paths
** Succesfully created experiment folder! **
** Created and modified temp lisp file **
** running lisp **
 
** we got here! done! **


# Section2: extracting data


load the experiment data we just run above

In [5]:
newest_experiment_history_path = sorted(glob(experiment_history_folder+'*'))[-1]+'/' # sorted the order of the list
selected_experiment_history_path = newest_experiment_history_path
dat_file_path = sorted(glob(selected_experiment_history_path+'experiment_output_data_folder/*'))[0]
all_song_dict = data_extractor.get_all_song_dict_from_dat(dat_file_path)


100%|██████████| 971/971 [00:00<00:00, 6048.11it/s]


choose which song you want to inspect by specifying the index 

# Single song feature extraction ( Jump to next section if you need batch extraction )

In [6]:
choose_song_index = 0                                             
song_dict_of_interest= list(all_song_dict.values())[choose_song_index] # one of the testing song result info.

- run this block to get ```aligned_surprise_with_onset```

In [19]:
#aligned_surprise_with_onset = data_extractor.get_aligned_surprise_with_onset_from_song_dict(song_dict_of_interest)

#print('aligned_surprise_with_onset: \n')
#print(aligned_surprise_with_onset)

- run this block to get ```surprise_sequence```

In [20]:
#surprise_sequence = data_extractor.get_surprise_from_song_dict(song_dict_of_interest)

#print('surprise_sequence: \n')
#print(surprise_sequence)

- run this block to get ```note_distribution```

In [21]:
#note_distribution = data_extractor.get_note_distribution_from_song_dict(song_dict_of_interest)

#print('note_distribution: \n')
#print(note_distribution)

- run this block to get ```onset_sequence```

In [22]:
#onset_sequence = data_extractor.get_onset_from_song_dict(song_dict_of_interest)

#print('onset_sequence: \n')
#print(onset_sequence)

# Batch feature extraction

In [115]:
def song_wise_extraction(song_dict,extraction_methods):
    single_song_data = []
    for extraction in extraction_methods:
        feature = extraction(song_dict).tolist()
        single_song_data.append(feature)
    #single_song_data = np.array(single_song_data,dtype=object)
    return single_song_data

def dataset_wise_extraction(all_song_dict,extraction_methods):
    all_song_data = []
    for index,song_dict in all_song_dict.items():
        single_song_data = song_wise_extraction(song_dict,extraction_methods)
        all_song_data.append(single_song_data)
    all_song_data = np.array(all_song_data,dtype=object)
    return all_song_data

features_method_name_dict = {
    'aligned_surprise_with_onset':data_extractor.get_aligned_surprise_with_onset_from_song_dict,
    'surprise': data_extractor.get_surprise_from_song_dict,
    'onset': data_extractor.get_onset_from_song_dict,
    'note_distribution':data_extractor.get_note_distribution_from_song_dict,
}

dict_access_keys = lambda dic,l:[dic[x] for x in l]

## Specify a list of feautures to extract (the available ones are in features_method_name_dict).

In [ ]:
my_choice_of_extraction = [
    'aligned_surprise_with_onset',
    'surprise',
    'onset',
    'note_distribution',
]

data_to_export = dataset_wise_extraction(all_song_dict,dict_access_keys(features_method_name_dict,my_choice_of_extraction)).T

# Export data of interest to file

In [ ]:
def export(path,data):
    for i,feature in enumerate(data_to_export):
        feature_name = list(features_method_name_dict.keys())[i]
        scipy.io.savemat(path+feature_name+'.mat', mdict={feature_name: np.array(feature)})

specify destination for exporting the file containing the data above

do the export

In [ ]:
your_file_path = selected_experiment_history_path
export(your_file_path,data_to_export)

# Export data of interest to file

In [128]:
def export(path,data):
    for i,feature in enumerate(data_to_export):
        feature_name = list(features_method_name_dict.keys())[i]
        scipy.io.savemat(path+feature_name+'.mat', mdict={feature_name: np.array(feature)})

specify destination for exporting the file containing the data above

do the export

In [129]:
your_file_path = selected_experiment_history_path
export(your_file_path,data_to_export)